# Aggregate Dataset for PSP



In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

In [2]:
dir_root = '../../data/modified_psp_data/'
dir_out = '../../data/modified_psp_data/'

path_train = os.path.join(dir_root, "df_agg_20230426_001200_01.csv")

df_agg = pd.read_csv(path_train)

In [3]:
len(list(df_agg.columns))

60

In [4]:
list_id = [
    'ith_sess',
    'session_id'
]

list_agg = [
    'navigate_click:0-4',
    'cutscene_click:0-4',
    'person_click:0-4',
    'object_click:0-4',
    'notification_click:0-4',
    'observation_click:0-4',
    'object_hover:0-4',
    'map_hover:0-4',
    'map_click:0-4',
    'checkpoint:0-4',
    'person_click:5-12',
    'navigate_click:5-12',
    'object_click:5-12',
    'object_hover:5-12',
    'cutscene_click:5-12',
    'map_hover:5-12',
    'notification_click:5-12',
    'map_click:5-12',
    'observation_click:5-12',
    'checkpoint:5-12',
    'navigate_click:13-22',
    'person_click:13-22',
    'cutscene_click:13-22',
    'object_click:13-22',
    'map_hover:13-22',
    'object_hover:13-22',
    'notification_click:13-22',
    'map_click:13-22',
    'observation_click:13-22',
    'checkpoint:13-22',
    'notebook_click:0-4',
    'notebook_click:5-12',
    'notebook_click:13-22'
]
list_agg.sort()

list_checkpoint_agg = [
    'elapsed_time:0-4',
    'event_index:0-4',
    'elapsed_time:5-12',
    'event_index:5-12',
    'elapsed_time:13-22',
    'event_index:13-22'
]
list_checkpoint_agg.sort()

list_q = [f"q{x}" for x in range(1, 19)]
list_q.append('num_correct')


list_cols = list_id + list_agg + list_checkpoint_agg + list_q

In [5]:
len(list_cols)

60

In [6]:
df_agg = df_agg[list_cols]

display(df_agg.shape)

df_agg.head()


(11779, 60)

,ith_sess,session_id,checkpoint:0-4,checkpoint:13-22,checkpoint:5-12,cutscene_click:0-4,cutscene_click:13-22,cutscene_click:5-12,map_click:0-4,map_click:13-22,...,q10,q11,q12,q13,q14,q15,q16,q17,q18,num_correct
0,0,20090312431273200,1.0,1,1,28,60,12,2,6,...,1,1,1,0,1,1,0,1,1,16
1,1,20090312433251036,1.0,1,1,36,65,11,3,45,...,0,0,1,0,1,0,1,0,1,10
2,2,20090314121766812,1.0,1,1,26,58,14,2,7,...,1,1,1,0,1,1,1,0,1,12
3,3,20090314363702160,1.0,1,1,32,76,11,2,12,...,1,1,1,0,1,0,0,1,1,15
4,4,20090314441803444,1.0,1,1,29,57,11,2,6,...,1,0,1,0,1,1,1,1,1,14


# Split into train and test data

In [14]:
import random

def create_train_test_splits(dir_out, df_data, test_split=0.2):
    list_unique_sessions = list(df_data['session_id'].unique())
    print(f"list_unique_sessions: {len(list_unique_sessions)}")
    
    len_data = len(list_unique_sessions)
    len_test = int(test_split * len_data)
    
    list_random_sessions = random.sample(list_unique_sessions, len_data)
    
    list_train_sessions = list_random_sessions[:-len_test]
    list_test_sessions = list_random_sessions[-len_test:]
    
    df_train = pd.DataFrame()
    df_train["session_id"] = list_train_sessions
    df_train["split_type"] = "train"
    
    df_test = pd.DataFrame()
    df_test["session_id"] = list_test_sessions
    df_test["split_type"] = "test"
    
    print(f"Train split: {df_train.shape}, Test split: {df_test.shape}")
    
    df_train_test_splits = pd.concat([df_train, df_test], axis=0)
    print(f"Combined len: {df_train_test_splits.shape}")
    
    split_out_path = os.path.join(dir_out, "df_train_test_splits.csv")
    
    df_train_test_splits.to_csv(split_out_path, index=False)
    print(f"Saved df_train_test_splits at: {split_out_path}")
    
    return df_train_test_splits
    

    
def get_train_test_splits_splits(df_agg):
    print(f"Combined len: {df_agg.shape}")
    
    split_out_path = os.path.join(dir_out, "df_train_test_splits.csv")
    df_train_test_splits = pd.read_csv(split_out_path)
    
    list_train_sessions = list(df_train_test_splits[df_train_test_splits["split_type"] == "train"]['session_id'])
    list_test_sessions = list(df_train_test_splits[df_train_test_splits["split_type"] == "test"]['session_id'])
 
    df_agg_train = df_agg[df_agg['session_id'].isin(list_train_sessions)].copy()
    df_agg_test = df_agg[df_agg['session_id'].isin(list_test_sessions)].copy()
    
    print(f"Train split: {df_agg_train.shape}, Test split: {df_agg_test.shape}")
    
    
    return df_agg_train, df_agg_test
    


In [15]:
# df_1 = create_train_test_splits(dir_out, df_agg)

df_agg_train, df_agg_test = get_train_test_splits_splits(df_agg)


Combined len: (11779, 60)
Train split: (9424, 60), Test split: (2355, 60)


# Statistical Analysis